In [40]:
import torch
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.profiler import profile, ProfilerActivity, record_function
from torch.utils.flop_counter import FlopCounterMode
import psutil
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import random

### Exercise 2.3

Use a pretrained language model from HuggingFace (we suggest DistilGPT2) and perform the following:
(a) Perplexity Analysis: Choose a short English paragraph (e.g., 3 to 5 sentences). Tokenize it and compute
its perplexity using the model’s log-likelihood output. Then compute the perplexity of a randomly shuffled
version of the same paragraph. Comment on the difference.
(b) Sampling Comparison: Using the prompt “Once upon a time”, generate continuations (length=500 tokens)
using:
• greedy decoding,
• temperature sampling with T ∈ {0, 0.3, 0.6, 0.9, 1.2, 1.5}.
Report the outputs and briefly compare their diversity and quality.

In [54]:
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

paragraph = "The calendar says that the first day of January is the start of a new year. But for me, it has always been September, when I put aside the sandy and slack routines of summer (hot dogs on the grill, takeout pizza on the beach) and stare down the months and school year ahead. What will I make for dinner tomorrow? What even goes in a lunchbox? It’s a time teeming with decisions to make, appointments to schedule and forms to fill; your calendar becomes your single source of truth."
paragraph_word = paragraph.split()
pargraph_sentence = paragraph.split(". ")
random.shuffle(paragraph_word)
random.shuffle(pargraph_sentence)
shuffled_paragraph_word = " ".join(paragraph_word)
shuffled_paragraph_sentence = ". ".join(pargraph_sentence)


inputs = tokenizer(paragraph, return_tensors="pt").to("cuda")
inputs_word = tokenizer(shuffled_paragraph_word, return_tensors="pt").to("cuda")
inputs_sentence = tokenizer(shuffled_paragraph_sentence, return_tensors="pt").to("cuda")

outputs = model(**inputs, labels=inputs['input_ids'])
outputs_word = model(**inputs_word, labels=inputs_word['input_ids'])
outputs_sentence = model(**inputs_sentence, labels=inputs_sentence['input_ids'])


In [55]:
print(f"Original Perplexity: {torch.exp(outputs.loss)}")
print(f"Sentence-Shuffled Perplexity: {torch.exp(outputs_sentence.loss)}")
print(f"Word-Shuffled Perplexity: {torch.exp(outputs_word.loss)}")


Original Perplexity: 63.6069221496582
Sentence-Shuffled Perplexity: 72.81319427490234
Word-Shuffled Perplexity: 1892.6475830078125


In [64]:
# generate 500 tokens
temperatures = [0.001, 0.3, 0.6, 0.9, 1.2, 1.5]
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id
input_ids = tokenizer("Once upon a time", return_tensors="pt").input_ids.to("cuda")

dictionary_outputs = {}
for temp in temperatures:
    generated = model.generate(input_ids, max_length=500, do_sample=True, temperature=temp)
    dictionary_outputs[temp] = tokenizer.decode(generated[0], skip_special_tokens=True)

greedy = model.generate(input_ids, max_length=500, do_sample=False)
dictionary_outputs["greedy"] = tokenizer.decode(greedy[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [65]:
for temperature, text in dictionary_outputs.items():
    print(f"Temperature: {temperature}\n")
    print(text)
    print("\n" + "="*80 + "\n")

Temperature: 0.001

Once upon a time of war, the United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a mi